In [1]:
!pwd
!ls

/home/ec2-user/SageMaker/histology-image-analysis/sagemaker-inference
mhist-predict.ipynb  model  src  test_locally.py


In [ ]:
# Test inference locally
!pip install -U -q -r src/requirements.txt
%run test_locally.py

In [2]:
!tar -czvf model.tar.gz model src

model/
model/.ipynb_checkpoints/
model/vit_onnx_model.onnx
src/
src/requirements.txt
src/.ipynb_checkpoints/
src/.ipynb_checkpoints/requirements-checkpoint.txt
src/.ipynb_checkpoints/__init__-checkpoint.py
src/.ipynb_checkpoints/inference-checkpoint.py
src/__pycache__/
src/__pycache__/__init__.cpython-310.pyc
src/__pycache__/inference.cpython-310.pyc
src/__init__.py
src/inference.py


In [4]:
# Upload model files to SageMaker default bucket
# !pip install -U sagemaker
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
dest = sagemaker_session.upload_data(path='model.tar.gz', bucket=bucket, key_prefix='mhist-vit-model')

print(f"Model files uploaded to: {dest}")

Model files uploaded to: s3://sagemaker-us-west-1-851725529671/mhist-vit-model/model.tar.gz


In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role() # arn:aws:iam::851725529671:role/SageMakerEx
model_data = 's3://mhist-artifacts-2024/vit_onnx_model.tar.gz'

model = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    framework_version='2.3.0', # PyTorch version
    py_version='py311'
)

predictor = model.deploy(
    instance_type='ml.m5.xlarge',
    initial_instance_count=1
)
response = predictor.predict({
    'bucket': 'mhist-streamlit-app',
    'key': 'images/original/MHIST_aah.png'
})
predictor.delete_endpoint()
response